In [2]:
import pandas as pd
from azapi import AZlyrics
import json
import requests
from IPython.display import clear_output
import re


In [3]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

api_key = get_file_contents('lastfm_key')

In [ ]:
artists = pd.read_csv("artists.csv")

In [ ]:
artists.head()

In [ ]:
artists.sort_values(by=['listeners_lastfm'])[:5000]
artists.head()

In [ ]:
f = open('artists.json')
data = json.load(f)
az_artists = pd.DataFrame(list(data.keys()),columns =['az_artist'])
az_artists.size

In [ ]:
merged = artists.merge(az_artists, how="inner",left_on="artist_mb", right_on="az_artist")
merged = merged[['mbid', 'artist_mb', 'artist_lastfm', 'tags_mb', 'az_artist']]
merged['done'] = False
merged = merged.drop_duplicates(subset=['artist_mb'])
merged.to_csv('artists-merged.csv', index=False);
print(merged.shape)
merged.head(20)

In [ ]:
api = AZlyrics('google')
song_df = pd.DataFrame(columns=['mbid', 'artist_mb', 'artist_lastfm', 'tags_mb', 'az_artist', 'song', 'lyrics', 'song_mbid'])
song_df.to_csv('songs.csv', index=False)

In [6]:
data = pd.read_csv("artists-merged.csv")
song_df = pd.read_csv("songs.csv")
good = []
bad = []


UPDATE THIS SCRIPT TO PULL MBID OF SONGS AND USE THAT TO GET IMAGES
for i,row in data.iterrows():
    if not row['done']:
        api.artist = row['az_artist']
        count = 0
        url = f"http://ws.audioscrobbler.com/2.0/?method=artist.gettoptracks&artist={row['artist_lastfm']}&api_key={api_key}&format=json&limit=30"
        r = requests.get(url = url)
        songs = None
        try:
            songs = r.json()['toptracks']['track']
        except Exception as e:
            print(e)
            print(api.artist, r.json())
            bad.append(row['az_artist'])
            break
        # print(songs)
        for i,song in enumerate(songs):
            clear_output(wait=True)
            print(f"parsing song number {i} with {count} fails")
            print(f'{len(good)} successed and {len(bad)} failed')
            if len(bad) <= 100:
                print(bad)
            api.title = song['name']
            if count >= 10:
                print(api.artist, api.title)
                bad.append(row['az_artist'])
                break
            # print(api.artist, api.title)
            try:
                lyrics = re.sub(r"\s+", " ", api.getLyrics(sleep=6))
                song_df.loc[len(song_df.index)] = list(row[:-1]) + [song['name'], lyrics, song['image'][2]["#text"], song['mbid']]
            except:
                count+=1
        
        song_df.to_csv('songs.csv', index=False)
        if count < 10:
            data.at[i,'done'] = True
            good.append(row['az_artist'])
        data.to_csv('artists-merged.csv', index=False)



parsing song number 8 with 8 fails
4 successed and 2 failed
['Lady Gaga', 'Daft Punk']


In [ ]:
l = ['Rihanna', 'Eminem', 'Kanye West', 'Nirvana', 'Muse', 'Queen', 'Foo Fighters', 'Linkin Park', 'Lady Gaga', 'Daft Punk', 'Green Day', 'Katy Perry', 'Oasis', 'Gorillaz', 'Michael Jackson', 'Maroon 5', 'Arctic Monkeys', 'U2', 'Drake', 'David Bowie', 'Britney Spears', 'Franz Ferdinand', 'Madonna', 'Pink Floyd', 'Weezer', 'Snow Patrol', 'Led Zeppelin', 'Beck', 'Aerosmith', 'Metallica', 'R.E.M.', 'Blur', 'Adele', 'Nickelback']
data.loc[data['az_artist'].isin(l), ['done']] = False
data.head(20)
data.to_csv('artists-merged.csv', index=False)

In [6]:
song_df = pd.read_csv("songs.csv")

In [5]:
api = AZlyrics()
api.artist='taylor swift'
api.getSongs()

https://www.azlyrics.com/t/taylorswift.html


{'Tim McGraw': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/timmcgraw.html'},
 'Picture To Burn': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/picturetoburn.html'},
 'Teardrops On My Guitar': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/teardropsonmyguitar.html'},
 'A Place In This World': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/aplaceinthisworld.html'},
 'Cold As You': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/coldasyou.html'},
 'The Outside': {'year': '2006',
  'album': 'Taylor Swift',
  'type': 'album',
  'url': 'http://www.azlyrics.com/lyrics/taylorswift/theoutside.html'},
 'Tied Together With A Smile': {'year': '2006',
  'al